# TrieSort

 Sorting algorithm based on trie data structure

> # Introduction
 We have comparison sorts like merge sort, insert sort, quick sort etc with best runtime complexity of O(nlogn) 
 and we have non comparison sorts like radix, bucket sort and counting sort but with limitations
 Today we are going to introduce TrieSort using the Trie data structure and using n comparisons. It's runtime 
 complexity is O(nlogk) where k is the len(str(max(nums))) where nums is the input array. The log part doesn't depend
 on the number of items in the list and it's to the base 10. It's not a comparison sort and uses index to avoid
 comparisons. Let's begin

> # The logic <br>
 Let's start from the simplest base case <br>
 Assume you have a dict (Trie is implemented using a recursive dict) and a list nums whose range is between 0 and 9
 Now if you keep inserting those elements in the dict with keys as the number itself you will have a dict like this

In [22]:
from random import randrange
import collections
# the dictionary we are gonna use to sort. 
sort_dict = collections.defaultdict(list)
for i in range(0, 9):
    el = randrange(10)
    sort_dict[el] = True # Value doesn't matter. Yet

In [23]:
sort_dict # we can see there were some duplicate numbers also. Don't worry we will take care of it later

defaultdict(list, {1: True, 8: True, 9: True, 0: True, 7: True, 2: True})

>   ## Pay attention. 
This is the trick. We know the dict have numbers from zero to nine. If we try every key from zero to nine some might be present others might not. We put everything that is present in a list in that order then we technically have our sorted list. Without any comparisons at all. 

Don't get too excited, we will need comparisons later.

In [24]:
sorted_list = [x for x in range(10) if sort_dict.get(x)]
sorted_list

[0, 1, 2, 7, 8, 9]

> # TADA!!!

Is it good? Someone just have one element to sort or two. Triesort will still create a dict and loop through zero to ninne to find out which digit that is. It might look like for small data set the comparison sorts are better. Too bad.

And alsw we sorted numbers who have only one digit each. Some guy might ask 

>  What if we had list of numbers upto 100 to sort?? 

 #### Opinion. 
 'Guy' should be gender neutral.

 #### Popular opinion.
 That guy could have asked for numbers upto 1000 to sort or a thousand million (billion!?). But let's take 100 for sake of simplicity